In [204]:
import pandas as pd
import constants 
import scipy 
import os

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

os.chdir("/specific/elkon/hagailevi/PRS/codebase")


In [209]:
discovery1="cimba_eur_brca1_oncoarray-5pcs" # "bcac_onco_eur-5pcs"
discovery2="cimba_eur_brca2_oncoarray-5pcs"
target="cimba_eur_brca2_icogs"
imp="impX_gen"
method="pt3"
th1="0.001"
th2="0.0000001"
calc_type="mono"

In [229]:
discovery1="bcac_onco_eur-5pcs" # "cimba_eur_brca1_oncoarray-5pcs" # "bcac_onco_eur-5pcs"
discovery2="cimba_eur_brca2_oncoarray-5pcs"
target="cimba_eur_brca2_icogs"
imp="impX_gen"
method="pt3"
th1="0.0000001" # "0.001"
th2="0.0000001"
calc_type="mono"

In [230]:
df1=pd.read_csv(f"{constants.PRSS_PATH}/{discovery1}_{target}/{imp}/prs.{calc_type}.{method}.{th1}.profile", \
                  delim_whitespace=True, index_col=0)
df1['SCORE']=scipy.stats.zscore(df1['SCORE'])
df1=df1['SCORE'].rename("SCORE1")


df2=pd.read_csv(f"{constants.PRSS_PATH}/{discovery2}_{target}/{imp}/prs.{calc_type}.{method}.{th2}.profile", \
                 delim_whitespace=True, index_col=0)

df2['SCORE']=scipy.stats.zscore(df2['SCORE'])
df2=df2['SCORE'].rename("SCORE2")

df3=pd.read_csv(f"{constants.DATASETS_PATH}/{target}/pheno", \
                 delim_whitespace=True, index_col=0)['label']-1


df_agg=pd.concat((df1,df2,df3), axis=1)

df_agg

,SCORE1,SCORE2,label
FID,,,
163545,2.879107,1.686478,1
163547,0.450950,1.686478,1
163548,-0.871404,-0.444210,0
163590,0.339417,1.733390,0
163596,-2.288219,-1.556461,1
...,...,...,...
205032,0.173963,1.733390,1
205034,-0.291742,0.621129,0
205037,0.207291,-0.491114,1


In [231]:
def calc_set(reg, X, y_true, p_high=99, p_low=1):

    y_proba=reg.predict_proba(X)[:,1]
    
    th_high=np.percentile(y_proba, p_high)
    th_low=np.percentile(y_proba, p_low)
#     print(y_proba)
#     print(th_high, max(y_proba))
#     print(th_low, min(y_proba))
#     print(len(X),len(y_true), len(y_true[y_proba<=th_low]))
    print(f"PRAUC: {average_precision_score(y_true, y_proba)} ({np.mean(y_true)})")
    print(f"Top {100-p_high}% accuracy: {np.round(sum(y_true[y_proba>=th_high])/len(y_true[y_proba>=th_high]),4)}")
    print(f"Bottom {p_low}% accuracy: {np.round((len(y_true[y_proba<=th_low])-sum(y_true[y_proba<=th_low]))/len(y_true[y_proba<=th_low]),4)}")

def calc_performance(X_train, X_test, y_train, y_test, reg):
   
    y_score_train=reg.predict(X_train)
    y_score_test=reg.predict(X_test)
    print("train:")
    calc_set(reg, X_train, y_train)
    print("test:")
    calc_set(reg, X_test, y_test)


In [232]:
y=df_agg['label']
X=df_agg[['SCORE1', 'SCORE2']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


print('Only score1:')
score_names=['SCORE1']
reg = LogisticRegression().fit(X_train[score_names], y_train)
calc_performance(X_train[score_names], X_test[score_names], y_train, y_test, reg)

print('Only score2:')
score_names=['SCORE2']
reg = LogisticRegression().fit(X_train[score_names], y_train)
calc_performance(X_train[score_names], X_test[score_names], y_train, y_test, reg)

print('Both scores:')
score_names=['SCORE1', 'SCORE2']
reg = LogisticRegression().fit(X_train, y_train)
calc_performance(X_train[score_names], X_test[score_names], y_train, y_test, reg)

Only score1:
train:
PRAUC: 0.6091438616907457 (0.5734265734265734)
Top 1% accuracy: 0.4
Bottom 1% accuracy: 0.6
test:
PRAUC: 0.6668271198551181 (0.660377358490566)
Top 1% accuracy: 0.6667
Bottom 1% accuracy: 0.0
Only score2:
train:
PRAUC: 0.6141361030924186 (0.5734265734265734)
Top 1% accuracy: 0.8889
Bottom 1% accuracy: 0.5085
test:
PRAUC: 0.6850248298956624 (0.660377358490566)
Top 1% accuracy: 0.7778
Bottom 1% accuracy: 0.4194
Both scores:
train:
PRAUC: 0.6222267468807554 (0.5734265734265734)
Top 1% accuracy: 0.8
Bottom 1% accuracy: 0.4
test:
PRAUC: 0.6889296316291218 (0.660377358490566)
Top 1% accuracy: 1.0
Bottom 1% accuracy: 0.0
